# pre

In [15]:
from mediocreatbest import auto


# lib

In [16]:
#@title Spatial
class Spatial:
    Degree = auto.typing.NewType('Degree', float)
    Radian = auto.typing.NewType('Radian', float)
    Meter = auto.typing.NewType('Meter', float)
    Kilometer = auto.typing.NewType('Kilometer', float)

    def __new__(
        cls,
        *,
        lat: Degree,
        lng: Degree,
        alt: Meter,
    ) -> tuple[Kilometer, Kilometer, Kilometer]:
        Degree = cls.Degree
        Radian = cls.Radian
        Meter = cls.Meter
        Kilometer = cls.Kilometer

        # Thanks https://gis.stackexchange.com/a/4148

        #> Note that "Lat/Lon/Alt" is just another name for spherical coordinates, and
        #> phi/theta/rho are just another name for latitude, longitude, and altitude.
        #> :) (A minor difference: altitude is usually measured from the surface of the
        #> sphere; rho is measured from the center -- to convert, just add/subtract the
        #> radius of the sphere.)
        phi: Radian = auto.np.radians(lat)
        theta: Radian = auto.np.radians(lng)

        # Thanks https://en.wikipedia.org/wiki/Earth_radius
        #> A globally-average value is usually considered to be 6,371 kilometres (3,959 mi)
        rho: Kilometer = 6_371 + alt / 1000.0

        #> x = math.cos(phi) * math.cos(theta) * rho
        x: Kilometer = auto.np.cos(phi) * auto.np.cos(theta) * rho

        #> y = math.cos(phi) * math.sin(theta) * rho
        y: Kilometer = auto.np.cos(phi) * auto.np.sin(theta) * rho

        #> z = math.sin(phi) * rho # z is 'up'
        z: Kilometer = auto.np.sin(phi) * rho

        #> (Note there's some slightly arbitrary choices here in what each axis means...
        #> you might want 'y' to point at the north pole instead of 'z', for example.)

        # I do :)
        y, z = z, y

        return x, y, z


# 1st

In [18]:
#@title Roads
@auto.functools.cache
def Roads(
    *,
    root: auto.pathlib.Path | auto.typing.Literal[...] = ...,
    path: auto.pathlib.Path | auto.typing.Literal[...] = ...,
    name: str = 'Road_Segment_-5693139270757726668.geojson',
    hash: str = '0a5d5bd0de4e8ec41528d04793d9f1e648d2f58322ab607a23ef06dbe7307aea',
) -> auto.geopandas.GeoDataFrame:
    strict = (
        True
        # False
    )

    if root is ...:
        root = auto.pathlib.Path.cwd() / 'data'
    
    if path is ...:
        path = root / name
    
    if strict:
        sha256 = auto.hashlib.sha256()
        with auto.tqdm.auto.tqdm(total=path.stat().st_size, unit='B', unit_scale=True) as pbar:
            with path.open('rb') as f:
                while (chunk := f.read(4096)):
                    pbar.update(len(chunk))
                    sha256.update(chunk)
            assert sha256.hexdigest() == hash, f'hash mismatch: {sha256.hexdigest()} != {hash}'

    gdf = auto.geopandas.read_file(
        path,
    )

    return gdf


def scope():
    roads = Roads()
    
    /display roads

/scope


100%|██████████| 669M/669M [00:02<00:00, 261MB/s] 


,NBR_TENN_CNTY,NBR_RTE,SPCL_CSE,CNTY_SEQ,RS_BEG_LOG_MLE,RS_END_LOG_MLE,LOG_MLE_LENGTH,NBR_URB_AREA,SPCL_SYS,SPCL_SYS_2,...,SUB_RTE,UPDT_ON,ID_NUMBER,SPCL_SYS_4,SPCL_SYS_5,SPCL_SYS_6,NBR_US_RTE_6,STATE_AID,OBJECTID,geometry
0,Davidson,0A961,0,1,0.000,0.036,0.036,Nashville,None,None,...,00,"Thu, 04 Nov 2010 09:29:04 GMT",190A961001,None,None,None,None,None,1,"LINESTRING (-86.58458 36.20733, -86.58459 36.2..."
1,Davidson,0A963,0,1,0.000,0.033,0.033,Nashville,None,None,...,00,"Thu, 04 Nov 2010 09:29:04 GMT",190A963001,None,None,None,None,None,2,"LINESTRING (-86.58417 36.209, -86.5842 36.2090..."
2,Davidson,0A967,0,1,0.000,0.039,0.039,Nashville,None,None,...,00,"Thu, 04 Nov 2010 09:29:04 GMT",190A967001,None,None,None,None,None,3,"LINESTRING (-86.59299 36.23505, -86.59309 36.2..."
3,Davidson,0A972,0,1,0.000,0.065,0.065,Nashville,None,None,...,00,"Thu, 04 Nov 2010 09:29:04 GMT",190A972001,None,None,None,None,None,4,"LINESTRING (-86.59386 36.24149, -86.5939 36.24..."
4,Davidson,0A974,0,1,0.117,0.167,0.050,Nashville,None,None,...,00,"Thu, 04 Nov 2010 09:29:04 GMT",190A974001,None,None,None,None,None,5,"LINESTRING (-86.59357 36.22338, -86.59371 36.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186635,Williamson,0A234,0,1,0.000,0.133,0.133,Nashville,None,None,...,00,"Thu, 09 Jul 2009 12:10:45 GMT",940A234001,None,None,None,None,None,186636,"LINESTRING (-86.88428 35.97796, -86.88429 35.9..."
186636,Williamson,0A744,0,1,0.000,0.524,0.524,Nashville,None,None,...,00,"Thu, 09 Jul 2009 12:10:45 GMT",940A744001,None,None,None,None,None,186637,"LINESTRING (-86.79204 35.9849, -86.79191 35.98..."
186637,Williamson,0A747,0,1,0.000,0.930,0.930,Nashville,None,None,...,00,"Wed, 22 May 2024 14:04:06 GMT",940A747001,None,None,None,None,None,186638,"LINESTRING (-86.81551 35.9884, -86.8154 35.988..."
186638,Williamson,0A748,0,1,0.000,0.366,0.366,Nashville,None,None,...,00,"Thu, 09 Jul 2009 12:10:45 GMT",940A748001,None,None,None,None,None,186639,"LINESTRING (-86.81223 35.98766, -86.81228 35.9..."


In [ ]:
#@title sanity check
def scope():
    roads = Roads()
    
    it = roads.iterrows()
    it = auto.tqdm.auto.tqdm(it, total=len(roads))
    
    for _, row in it:
        geometry = row['geometry']
        
        if geometry is None:
            continue
        
        # is linestring or multilinestring
        assert any([
            isinstance(geometry, auto.shapely.geometry.linestring.LineString),
            isinstance(geometry, auto.shapely.geometry.multilinestring.MultiLineString),
        ]), type(geometry)

/scope


100%|██████████| 186640/186640 [00:13<00:00, 14072.26it/s]


In [19]:
#@title make OSPRay data
def scope():
    roads = Roads()
    
    def scope():
        N = 0
        
        it = roads.iterrows()
        it = auto.tqdm.auto.tqdm(it, total=len(roads), leave=False)
        
        for _, row in it:
            geometry = row['geometry']
            if geometry is None:
                continue

            if isinstance(geometry, auto.shapely.geometry.linestring.LineString):
                N += len(geometry.coords)
            elif isinstance(geometry, auto.shapely.geometry.multilinestring.MultiLineString):
                N += sum(len(line.coords) for line in geometry.geoms)

        return N
    N = scope()
    
    verts = auto.np.full(N, 0.37373737, dtype=[
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('r', 'f4'),
    ])
    lines = auto.np.full(N, auto.np.iinfo('u4').max, dtype=[
        ('a', 'u4'),
    ])
    VERT = iter(auto.itertools.count())
    LINE = iter(auto.itertools.count())
    RADIUS = 0.1  # km
    
    it = roads.iterrows()
    it = auto.tqdm.auto.tqdm(it, total=len(roads))
    
    for _, row in it:
        geometry = row['geometry']
        if geometry is None:
            continue
        
        if isinstance(geometry, auto.shapely.geometry.linestring.LineString):
            geometry = [geometry]
        elif isinstance(geometry, auto.shapely.geometry.multilinestring.MultiLineString):
            geometry = geometry.geoms
        else:
            assert False, type(geometry)
        
        for geometry in geometry:
            indices = []
            for lng, lat in geometry.coords:
                x, y, z = Spatial(
                    lat=lat,
                    lng=lng,
                    alt=0,
                )
                
                verts[(i := next(VERT))] = (x, y, z, RADIUS)
                indices.append(i)

            for a, b in auto.itertools.pairwise(indices):
                lines[next(LINE)] = a

    verts = verts[:next(VERT)]
    lines = lines[:next(LINE)]
    
    root = auto.pathlib.Path.cwd() / 'data' / 'Roads'
    root.mkdir(exist_ok=True, parents=True)
    
    with (path := root / 'OSPGeometry.curve.vertex.position_radius.vec4f[].bin').open('wb') as f:
        f.write(verts.tobytes())
    print(f'Wrote {path.stat().st_size:,d} bytes to {path}')
    
    with (path := root / 'OSPGeometry.curve.index.uint[].bin').open('wb') as f:
        f.write(lines.tobytes())
    print(f'Wrote {path.stat().st_size:,d} bytes to {path}')

/scope


100%|██████████| 186640/186640 [03:10<00:00, 981.16it/s] 


Wrote 230,309,488 bytes to /home/thobson2/src/Sunrise/data/Roads/OSPGeometry.curve.vertex.position_radius.vec4f[].bin
Wrote 56,834,996 bytes to /home/thobson2/src/Sunrise/data/Roads/OSPGeometry.curve.index.uint[].bin
